In [17]:
%load_ext autoreload
%autoreload 2
from sklearn.model_selection import ParameterGrid
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn import metrics
from notebooks.sentiment.util import build_pipeline, print_eval, preprocess_tweets
from sentiment.new_data import InterTASSAugmented
from sentiment.tass import InterTASSReader
from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
# reader = InterTASSAugmented("2augmented_data_heu", ratio=0.9)  # Class to use augmented data
# X_train, y_train = reader.Xy()
train = "intertass-ES-train-tagged.xml"
train = InterTASSReader(train)
X_train, y_train = list(train.X()), list(train.y())


corpus = "intertass-ES-development-tagged.xml"
dev = InterTASSReader(corpus)
X_dev, y_dev = list(dev.X()), list(dev.y())

pipeline = build_pipeline()
pipeline.fit(X_train, y_train)

/home/gosen/.local/share/virtualenvs/PLN-2019-N1iczKTA/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/gosen/.local/share/virtualenvs/PLN-2019-N1iczKTA/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Pipeline(memory=None,
     steps=[('feats', FeatureUnion(n_jobs=None,
       transformer_list=[('vect', TfidfVectorizer(analyzer='char_wb', binary=True, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.95, max_features=None, min_df=5,
        ngr...penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False))])

In [52]:
print_eval(pipeline, X_dev, y_dev, )

accuracy	0.56

              precision    recall  f1-score   support

           n       0.58      0.77      0.66       219
           p       0.20      0.07      0.11        69
        none       0.24      0.10      0.14        62
         neu       0.61      0.65      0.63       156

   micro avg       0.56      0.56      0.56       506
   macro avg       0.41      0.40      0.38       506
weighted avg       0.50      0.56      0.51       506

[[168  11   9  31]
 [ 40   5   4  20]
 [ 37   5   6  14]
 [ 44   4   6 102]]


In [53]:
pipeline.steps

[('feats', FeatureUnion(n_jobs=None,
         transformer_list=[('vect', TfidfVectorizer(analyzer='char_wb', binary=True, decode_error='strict',
          dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=0.95, max_features=None, min_df=5,
          ngram_range=(1, 6), norm='l2',
          preprocessor=<...      tokenizer=<function word_tokenize at 0x7fb6a6d0ed90>, use_idf=True,
          vocabulary=None))],
         transformer_weights=None)),
 ('clf', LogisticRegression(C=1.0, class_weight='balanced', dual=False,
            fit_intercept=True, intercept_scaling=1, max_iter=100,
            multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
            solver='warn', tol=0.0001, verbose=0, warm_start=False))]

### Inspeccion de modelos:

**El modelo a analizar fue entrenado usando regresion logistica y la union
de dos clasificadores, uno que usó "word" como _analyzer_ y el otro uso "char".
Veamos cuales son los 10 features con mas peso positivo y mas peso negativo para cada clase:**

In [61]:
vect = pipeline.named_steps['feats']
clf = pipeline.named_steps['clf']
from sentiment.analysis import print_maxent_features
print_maxent_features(vect, clf, n=10, prettify=True)

N:
	! ! !   A guapa rt s! rac eli 1 ([-0.83341789 -0.71006416 -0.69528888 -0.39891781 -0.39116628 -0.35919981
 -0.35230278 -0.34752441 -0.34490157 -0.34457249])
	odi me  no od  me  triste  n  no no   no  ([0.45335267 0.49109051 0.49409086 0.50665793 0.54603738 0.59735805
 0.66252618 0.77205035 0.90183338 0.91858853])
NEU:
	j ap ño am ?  y   L L uc ol ([-0.54014245 -0.49413524 -0.46230685 -0.44611203 -0.42453641 -0.42350058
 -0.42126836 -0.41817165 -0.40529656 -0.40098482])
	ser cereal cereal hacen tio Estoy nerviosa Lo hecho , dicho  lo  plan bonito serio viejas ([0.59900622 0.59900622 0.61462497 0.63610812 0.64944056 0.65468827
 0.6589533  0.73702051 0.95766574 1.04788207])
NONE:
	 mu mu  m  me á  no   me   muy  muy  uy  ([-0.63417026 -0.62699317 -0.568688   -0.56681019 -0.54344921 -0.51474655
 -0.50087856 -0.48762919 -0.48762919 -0.48095096])
	ct 0  stra si Votado abstracto juntos ?  ? ? ([0.5468255  0.55525434 0.57163574 0.58832631 0.78823053 0.91056084
 0.92608845 0.98436306 1.0248

**En aspectos generales creo que la mayoria de los features tienen sentido, pero dado que se usan ambos analizadores y no solo el que toma palabras vemos que aparecen varios conjuntos de letras que no significan una palabra en sí, como "rt", "mu", "me", "uy", "ct" etc. entonces, esto complica el analisis.
Pero si vamos a los casos mas claros, por ejemplo para la clase los positivos se ve claramente como la palabra "no" y la palabra "triste" son los features que mas desfavorecen la clase, lo que es correcto.
Después por ejemplo, en los neutros, vemos palabras como "cereal", "ser", "hecho", "nerviosa" que en si estas palabras, no tienen una carga de sentimiento por si mismas, si no que dependen del contexto en el cual son utilizadas.
Como feature, engañoso o incorrecto, pueden ser los que son letras que no se sabe a que palabra refieren o por que tienen tanto peso, como que "eli" tiene peso negativo en la clase negativa, pero a priori no se sabe a que palabra corresponde "eli".
Otro punto importante que vale la pena observar, es el gran peso que tienen los signos de exclamacion en la clase positiva, o los signos de interrogacion en la clase "NONE".**